### **KLUE 텍스트 데이터 전처리**
- 빈값 NULL로 처리해서 제거
- 딕셔너리 값 변경
- 정규표현식으로 한글만 남겨두기
- 띄어쓰기 교정 (pykospacing 패키지 활용)
- 토큰화 (토크나이저 비교)



In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# Initial Setting (font, display)
!sudo apt-get install -y fonts-nanum # 글씨체 다운로드
!sudo fc-cache -fv # 폰트 리스트 갱신
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
import matplotlib
get_ipython().run_line_magic('config', "InlineBackend.figure_format='retina'") # 화질 보정

import os
import re
import pandas as pd
import numpy as np
import seaborn as sns

# NLP packages
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import nltk
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize
print('Ready')

In [ ]:
!pip3 install kss
!pip3 install git+https://github.com/haven-jeon/PyKoSpacing.git  # 한글 띄어쓰기(PyKoSpacing)

#### 1) 데이터 로드

In [68]:
klue_1 = pd.read_excel('/content/drive/MyDrive/klue_dataset/기다연_크롤링.xlsx')
klue_2 = pd.read_excel('/content/drive/MyDrive/klue_dataset/박주영_크롤링.xlsx')
klue_3 = pd.read_excel('/content/drive/MyDrive/klue_dataset/홍석민_크롤링.xlsx')
klue_4 = pd.read_excel('/content/drive/MyDrive/klue_dataset/금지헌_크롤링.xlsx')
klue = pd.concat([klue_1, klue_2, klue_3, klue_4], axis=0)
klue.head(5)

,class_name,professor_name,evaluation
0,물성전자공학(영강),김수진,"- 영강이지만 거의 5:5비율로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- PPT를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습문제에서 출제됨.\n교재를 찾아보면 같은 문제가 있을 것임.\n\n- 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나, 대부분의 문제가 객관식으로 출제됨.\n- 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효숫자에 신경쓰지 않고 풀 수 있음.\n- 풀이과정을 보시는지는 모르겠지만, 객관식이어서 각 답안이 맞으면 만점, 틀리면 0점임.\n- 주관식이나 서술형도 있지만 객관식이 대부분임.\n- 그러나 서술형 문제의 배점이 어마무시함. 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음.\n- 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분점수를 후하게 주시는 것 같음.\n- 중간고사 때는 중간고사 샘플자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음. 다만 기말고사에는 따로 없었음.\n- 나만 그런지는 모르겠으나,, 체감 난이도는 높았음. 과제보다는 난이도가 훨씬 쉽긴 함. 기말고사는 비대면 시험으로 진행되다 보니\n작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나, 체감 난이도는 높았음 ㅜ\n- 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면, 시험 기간에 따로 매달리지는 않아도 될 정도의 난이도로 나왔음.\n- 나머지는 수업자료나 과제에서 사용한 공식들을 익히고 가면 충분히 고득점을 노려볼 수 있음.\n\n- 학점은 항상 잘 주신다고 소문이 나 있음.\n\n- 예전에 전기회로1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안되어 집중을 못해서 잘 몰랐었는데..\n강의력이 정말정말 좋으심. 수업을 듣는데 흥미가 생김.\n- 과목 특성 상 고등학교 때 물리2 화학2 공부했다면 친숙한 내용임. 아니더라도 1학년 때 배우는 일반물리학/일반화학 내용이라 부담은 적음.\n양자역학도 배우긴 하지만 깊숙히 들어가지는 않아서 괜찮음.\n- 추천하고 싶은 강의임."
1,물성전자공학(영강),김수진,김수진 교수님의 물성전자공학은 사이버 강의였음에도 강의력이 좋으셔서 그런지 수월하게 했습니다. 과제는 3~4번 정도 나왔고 시험은 중간고사 때 조금 생각해야하는 문제 하나가 있는 것을 제외하면 전반적으로 쉬웠습니다. 선택하면 후회는 없는 교수님이라고 생각합니다. 총 평쁠 20점 정도로 에이쁠 받았는데 적당히 평쁠하면 받을 수 있는 걸로 알고 있습니다. 이 강의를 들으면서 이번에 들은 강의 중에 제일 재밌고 유익하다고 생각했습니다.
2,물성전자공학(영강),김수진,"강의도 너무 열정적으로 잘하시고, 학점도 두말하면 잔소리입니다 ㅠㅠ 그래도 강의 자료는 꼼꼼히 보고 시험 응시하시고 주변에서 너무 꿀이다 이런 말 많지만 그래도 공부는 나름 부지런히 하셔야 합니다!\n출쳌도 열심히하세요!"
3,물성전자공학(영강),김수진,"물성 강의의 1타강사이시다.\n영강이지만 영어 설명 이후 종종 한국어로 다시 한번 설명해주신다.\n평가 방식은 중간/기말/과제이며, 시험은 그렇게 어렵지 않다.\n출첵은 거의 하지 않으신다.\n물성 잘 모르면 이거만 기억하면 된다.\n\n""김수진"""
4,물성전자공학(영강),김수진,정말 최고의 수업이었습니다 진짜로 확실히 내 짧은 2년간의 수업중에는 최고였습니다 학점을 대박잘주셨습니다 역시 갓수진


In [9]:
klue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1696 entries, 0 to 474
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   class_name      1696 non-null   object
 1   professor_name  1696 non-null   object
 2   evaluation      1696 non-null   object
dtypes: object(3)
memory usage: 53.0+ KB


#### 2) 빈값 NULL값 처리

In [10]:
# 1) check NULL value
# NULl값이 없더라도 빈 값 유무 확인하기 위해 모든 빈 값을 NULL로 변환하고, 다시 NULL값이 있는지 확인
klue['evaluation'].replace("", float("NaN"), inplace=True)
print(klue['evaluation'].isnull().values.any())

False


In [11]:
klue['evaluation'].dropna(inplace=True)

In [12]:
# 길이 확인
print('총 샘플 수 :', len(klue))

총 샘플 수 : 1696


#### 3) 딕셔너리값 변경

In [69]:
# 영문은 소문자로 변경
def lower(text):
    result = text.lower()
    return(result)

lower_corpus = []
for sent in klue['evaluation']:
    lower_corpus.append(lower(sent))

klue['evaluation_low'] = lower_corpus
klue.head(3)

,class_name,professor_name,evaluation,evaluation_low
0,물성전자공학(영강),김수진,"- 영강이지만 거의 5:5비율로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- PPT를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습문제에서 출제됨.\n교재를 찾아보면 같은 문제가 있을 것임.\n\n- 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나, 대부분의 문제가 객관식으로 출제됨.\n- 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효숫자에 신경쓰지 않고 풀 수 있음.\n- 풀이과정을 보시는지는 모르겠지만, 객관식이어서 각 답안이 맞으면 만점, 틀리면 0점임.\n- 주관식이나 서술형도 있지만 객관식이 대부분임.\n- 그러나 서술형 문제의 배점이 어마무시함. 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음.\n- 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분점수를 후하게 주시는 것 같음.\n- 중간고사 때는 중간고사 샘플자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음. 다만 기말고사에는 따로 없었음.\n- 나만 그런지는 모르겠으나,, 체감 난이도는 높았음. 과제보다는 난이도가 훨씬 쉽긴 함. 기말고사는 비대면 시험으로 진행되다 보니\n작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나, 체감 난이도는 높았음 ㅜ\n- 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면, 시험 기간에 따로 매달리지는 않아도 될 정도의 난이도로 나왔음.\n- 나머지는 수업자료나 과제에서 사용한 공식들을 익히고 가면 충분히 고득점을 노려볼 수 있음.\n\n- 학점은 항상 잘 주신다고 소문이 나 있음.\n\n- 예전에 전기회로1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안되어 집중을 못해서 잘 몰랐었는데..\n강의력이 정말정말 좋으심. 수업을 듣는데 흥미가 생김.\n- 과목 특성 상 고등학교 때 물리2 화학2 공부했다면 친숙한 내용임. 아니더라도 1학년 때 배우는 일반물리학/일반화학 내용이라 부담은 적음.\n양자역학도 배우긴 하지만 깊숙히 들어가지는 않아서 괜찮음.\n- 추천하고 싶은 강의임.","- 영강이지만 거의 5:5비율로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- ppt를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습문제에서 출제됨.\n교재를 찾아보면 같은 문제가 있을 것임.\n\n- 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나, 대부분의 문제가 객관식으로 출제됨.\n- 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효숫자에 신경쓰지 않고 풀 수 있음.\n- 풀이과정을 보시는지는 모르겠지만, 객관식이어서 각 답안이 맞으면 만점, 틀리면 0점임.\n- 주관식이나 서술형도 있지만 객관식이 대부분임.\n- 그러나 서술형 문제의 배점이 어마무시함. 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음.\n- 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분점수를 후하게 주시는 것 같음.\n- 중간고사 때는 중간고사 샘플자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음. 다만 기말고사에는 따로 없었음.\n- 나만 그런지는 모르겠으나,, 체감 난이도는 높았음. 과제보다는 난이도가 훨씬 쉽긴 함. 기말고사는 비대면 시험으로 진행되다 보니\n작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나, 체감 난이도는 높았음 ㅜ\n- 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면, 시험 기간에 따로 매달리지는 않아도 될 정도의 난이도로 나왔음.\n- 나머지는 수업자료나 과제에서 사용한 공식들을 익히고 가면 충분히 고득점을 노려볼 수 있음.\n\n- 학점은 항상 잘 주신다고 소문이 나 있음.\n\n- 예전에 전기회로1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안되어 집중을 못해서 잘 몰랐었는데..\n강의력이 정말정말 좋으심. 수업을 듣는데 흥미가 생김.\n- 과목 특성 상 고등학교 때 물리2 화학2 공부했다면 친숙한 내용임. 아니더라도 1학년 때 배우는 일반물리학/일반화학 내용이라 부담은 적음.\n양자역학도 배우긴 하지만 깊숙히 들어가지는 않아서 괜찮음.\n- 추천하고 싶은 강의임."
1,물성전자공학(영강),김수진,김수진 교수님의 물성전자공학은 사이버 강의였음에도 강의력이 좋으셔서 그런지 수월하게 했습니다. 과제는 3~4번 정도 나왔고 시험은 중간고사 때 조금 생각해야하는 문제 하나가 있는 것을 제외하면 전반적으로 쉬웠습니다. 선택하면 후회는 없는 교수님이라고 생각합니다. 총 평쁠 20점 정도로 에이쁠 받았는데 적당히 평쁠하면 받을 수 있는 걸로 알고 있습니다. 이 강의를 들으면서 이번에 들은 강의 중에 제일 재밌고 유익하다고 생각했습니다.,김수진 교수님의 물성전자공학은 사이버 강의였음에도 강의력이 좋으셔서 그런지 수월하게 했습니다. 과제는 3~4번 정도 나왔고 시험은 중간고사 때 조금 생각해야하는 문제 하나가 있는 것을 제외하면 전반적으로 쉬웠습니다. 선택하면 후회는 없는 교수님이라고 생각합니다. 총 평쁠 20점 정도로 에이쁠 받았는데 적당히 평쁠하면 받을 수 있는 걸로 알고 있습니다. 이 강의를 들으면서 이번에 들은 강의 중에 제일 재밌고 유익하다고 생각했습니다.
2,물성전자공학(영강),김수진,"강의도 너무 열정적으로 잘하시고, 학점도 두말하면 잔소리입니다 ㅠㅠ 그래도 강의 자료는 꼼꼼히 보고 시험 응시하시고 주변에서 너무 꿀이다 이런 말 많지만 그래도 공부는 나름 부지런히 하셔야 합니다!\n출쳌도 열심히하세요!","강의도 너무 열정적으로 잘하시고, 학점도 두말하면 잔소리

In [70]:
# 딕셔너리내 값 대체
grades = {
"A" : "에이" , "A0" : "에이" , "A+" : "에이플러스", "에이쁠" : "에이플러스", "에이플" : "에이플러스", "에이플러스" : "에이플러스", "B" : "비" ,
"B0" : "비", "B+" : "비플러스", "비쁠" : "비플러스", "비플" : "비플러스", "비 플러스" : "비플러스", "C0" : "씨",
"C" : "씨", "C+" : "씨플러스", "씨쁠" : "씨플러스", "씨플" : "씨플러스", "씨 플러스" : "씨플러스", "에제": "에이", "에쁠":"에이플러스"
}

replace_words = {
"take home exam": "비대면 시험", 
"take-home exam": "비대면 시험", 
"ppt": "자료", "pdf" : "자료",
"ㅜㅜ":"","ㅠㅠ":"","ㅜ":"","ㅠ":"", "5:5비율로":"반반으로",
"t/f": "참 혹은 거짓", "캠": "카메라", "cpa, 씨파": "공인회계사시험", "o": "함",  "x" : "안함", "칼절평": "절대평가", "절평": "절대평가", "상평": "상대평가",
"굿": "좋다", "잼관": "재무관리", "갓" : "매우 좋으신", "줌, zoom, collaborate, 콜라보레이트, 칼투라, kaltura" : "온라인 강의 플랫폼", "미디언": "중위수", "q1": "제1사분위수", "q3": "제3사분위수",
"꿀강": "편한 강의", "띵강": "명강의", "헬강": "힘든 강의", "capm": "자본자산가격결정모형", 
"치팅시트" : "참고 노트", "치팅페이퍼" : "참고 노트", "오픈북":"참고 노트", "오픈 북":"참고 노트", "open book":"참고 노트",
"flipped class" : "쌍방향 소통 수업", "k-mooc" : "온라인 수업", "mooc":"온라인 수업",
"에이": "좋은 학점", "에이플러스": "좋은 학점", "비": "평범한 학점", "비플러스": "평범한 학점", "씨": "나쁜 학점", "씨플러스": "나쁜 학점", 
"peer review": "동료 평가", "case study": "사례 분석", "ipo": "기업평가", 
"r" : "통계 프로그램", "spss" : "통계 프로그램", "sas" : "통계 프로그램", "sql" : "통계 프로그램", "r studio" : "통계 프로그램",
"hbr" : "경영학 잡지", "%": "퍼센트", "프로":"퍼센트", "퍼":"퍼센트",
"ot, 오티": "오리엔테이션", "pg" : "쪽", "page":"쪽",
"essay" : "에세이", "youtube" : "유튜브", "q&a" : "질의응답", "qna":"질의응답",
"p/f" : "합격 또는 불합격", "페논페": "합격 또는 불합격", "pass": "통과", 
"5:5" : "반반",  "1학년" : "일학년",  "2학년" : "이학년",  "3학년" : "삼학년",  "4학년" : "사학년", 
"테뱅": "기출문제", "테스트뱅크" : "기출문제",
"비추": "추천 안함", "스윗": "친절한",
"워크로드": "공부량", "work load":"공부량",
}

In [71]:
grades_temp = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k, v in grades.items()}
replace_temp = {r'(\b){}(\b)'.format(k):r'\1{}\2'.format(v) for k, v in replace_words.items()}

In [73]:
klue['evaluation_dict'] = klue['evaluation_low'].replace(grades_temp, regex=True)
klue['evaluation_dict'] = klue['evaluation_dict'].replace(replace_temp, regex=True)

#### 4) 정규표현식으로 한글만 남겨두기

In [75]:
# 2) 정규표현식 (영어/기호/문자 제거)
# 텍스트 정제 함수: 한글과 숫자 이외의 문자는 모두 제거
def text_cleaning(text):
    hangulenglish = re.compile('[^ \!0-9\u3131-\u3163\uac00-\ud7a3]+')  # 한글과 숫자 제외한 모든 글자
    result = hangulenglish.sub('', str(text))  # 해당 글자 공백으로 대체
    return(result)

In [76]:
cleaned_corpus = []
for sent in klue['evaluation_dict']:
    cleaned_corpus.append(text_cleaning(sent))

klue['evaluation_kor'] = cleaned_corpus
klue.head(3)

,class_name,professor_name,evaluation,evaluation_low,evaluation_dict,evaluation_kor
0,물성전자공학(영강),김수진,"- 영강이지만 거의 5:5비율로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- PPT를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습문제에서 출제됨.\n교재를 찾아보면 같은 문제가 있을 것임.\n\n- 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나, 대부분의 문제가 객관식으로 출제됨.\n- 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효숫자에 신경쓰지 않고 풀 수 있음.\n- 풀이과정을 보시는지는 모르겠지만, 객관식이어서 각 답안이 맞으면 만점, 틀리면 0점임.\n- 주관식이나 서술형도 있지만 객관식이 대부분임.\n- 그러나 서술형 문제의 배점이 어마무시함. 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음.\n- 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분점수를 후하게 주시는 것 같음.\n- 중간고사 때는 중간고사 샘플자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음. 다만 기말고사에는 따로 없었음.\n- 나만 그런지는 모르겠으나,, 체감 난이도는 높았음. 과제보다는 난이도가 훨씬 쉽긴 함. 기말고사는 비대면 시험으로 진행되다 보니\n작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나, 체감 난이도는 높았음 ㅜ\n- 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면, 시험 기간에 따로 매달리지는 않아도 될 정도의 난이도로 나왔음.\n- 나머지는 수업자료나 과제에서 사용한 공식들을 익히고 가면 충분히 고득점을 노려볼 수 있음.\n\n- 학점은 항상 잘 주신다고 소문이 나 있음.\n\n- 예전에 전기회로1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안되어 집중을 못해서 잘 몰랐었는데..\n강의력이 정말정말 좋으심. 수업을 듣는데 흥미가 생김.\n- 과목 특성 상 고등학교 때 물리2 화학2 공부했다면 친숙한 내용임. 아니더라도 1학년 때 배우는 일반물리학/일반화학 내용이라 부담은 적음.\n양자역학도 배우긴 하지만 깊숙히 들어가지는 않아서 괜찮음.\n- 추천하고 싶은 강의임.","- 영강이지만 거의 5:5비율로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- ppt를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습문제에서 출제됨.\n교재를 찾아보면 같은 문제가 있을 것임.\n\n- 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나, 대부분의 문제가 객관식으로 출제됨.\n- 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효숫자에 신경쓰지 않고 풀 수 있음.\n- 풀이과정을 보시는지는 모르겠지만, 객관식이어서 각 답안이 맞으면 만점, 틀리면 0점임.\n- 주관식이나 서술형도 있지만 객관식이 대부분임.\n- 그러나 서술형 문제의 배점이 어마무시함. 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음.\n- 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분점수를 후하게 주시는 것 같음.\n- 중간고사 때는 중간고사 샘플자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음. 다만 기말고사에는 따로 없었음.\n- 나만 그런지는 모르겠으나,, 체감 난이도는 높았음. 과제보다는 난이도가 훨씬 쉽긴 함. 기말고사는 비대면 시험으로 진행되다 보니\n작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나, 체감 난이도는 높았음 ㅜ\n- 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면, 시험 기간에 따로 매달리지는 않아도 될 정도의 난이도로 나왔음.\n- 나머지는 수업자료나 과제에서 사용한 공식들을 익히고 가면 충분히 고득점을 노려볼 수 있음.\n\n- 학점은 항상 잘 주신다고 소문이 나 있음.\n\n- 예전에 전기회로1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안되어 집중을 못해서 잘 몰랐었는데..\n강의력이 정말정말 좋으심. 수업을 듣는데 흥미가 생김.\n- 과목 특성 상 고등학교 때 물리2 화학2 공부했다면 친숙한 내용임. 아니더라도 1학년 때 배우는 일반물리학/일반화학 내용이라 부담은 적음.\n양자역학도 배우긴 하지만 깊숙히 들어가지는 않아서 괜찮음.\n- 추천하고 싶은 강의임.","- 영강이지만 거의 반반으로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- ppt를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 

#### 5) pykospacing으로 띄어쓰기 교정

In [77]:
# 3) pykospacing으로 띄어쓰기 교정
from pykospacing import Spacing
spacing = Spacing()

spaced_corpus = []
for sent in klue['evaluation_kor']:
    spaced_corpus.append(spacing(sent))

klue['evaluation_spaced'] = spaced_corpus
klue.head(3)

,class_name,professor_name,evaluation,evaluation_low,evaluation_dict,evaluation_kor,evaluation_spaced
0,물성전자공학(영강),김수진,"- 영강이지만 거의 5:5비율로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- PPT를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습문제에서 출제됨.\n교재를 찾아보면 같은 문제가 있을 것임.\n\n- 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나, 대부분의 문제가 객관식으로 출제됨.\n- 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효숫자에 신경쓰지 않고 풀 수 있음.\n- 풀이과정을 보시는지는 모르겠지만, 객관식이어서 각 답안이 맞으면 만점, 틀리면 0점임.\n- 주관식이나 서술형도 있지만 객관식이 대부분임.\n- 그러나 서술형 문제의 배점이 어마무시함. 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음.\n- 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분점수를 후하게 주시는 것 같음.\n- 중간고사 때는 중간고사 샘플자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음. 다만 기말고사에는 따로 없었음.\n- 나만 그런지는 모르겠으나,, 체감 난이도는 높았음. 과제보다는 난이도가 훨씬 쉽긴 함. 기말고사는 비대면 시험으로 진행되다 보니\n작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나, 체감 난이도는 높았음 ㅜ\n- 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면, 시험 기간에 따로 매달리지는 않아도 될 정도의 난이도로 나왔음.\n- 나머지는 수업자료나 과제에서 사용한 공식들을 익히고 가면 충분히 고득점을 노려볼 수 있음.\n\n- 학점은 항상 잘 주신다고 소문이 나 있음.\n\n- 예전에 전기회로1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안되어 집중을 못해서 잘 몰랐었는데..\n강의력이 정말정말 좋으심. 수업을 듣는데 흥미가 생김.\n- 과목 특성 상 고등학교 때 물리2 화학2 공부했다면 친숙한 내용임. 아니더라도 1학년 때 배우는 일반물리학/일반화학 내용이라 부담은 적음.\n양자역학도 배우긴 하지만 깊숙히 들어가지는 않아서 괜찮음.\n- 추천하고 싶은 강의임.","- 영강이지만 거의 5:5비율로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- ppt를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습문제에서 출제됨.\n교재를 찾아보면 같은 문제가 있을 것임.\n\n- 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나, 대부분의 문제가 객관식으로 출제됨.\n- 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효숫자에 신경쓰지 않고 풀 수 있음.\n- 풀이과정을 보시는지는 모르겠지만, 객관식이어서 각 답안이 맞으면 만점, 틀리면 0점임.\n- 주관식이나 서술형도 있지만 객관식이 대부분임.\n- 그러나 서술형 문제의 배점이 어마무시함. 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음.\n- 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분점수를 후하게 주시는 것 같음.\n- 중간고사 때는 중간고사 샘플자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음. 다만 기말고사에는 따로 없었음.\n- 나만 그런지는 모르겠으나,, 체감 난이도는 높았음. 과제보다는 난이도가 훨씬 쉽긴 함. 기말고사는 비대면 시험으로 진행되다 보니\n작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나, 체감 난이도는 높았음 ㅜ\n- 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면, 시험 기간에 따로 매달리지는 않아도 될 정도의 난이도로 나왔음.\n- 나머지는 수업자료나 과제에서 사용한 공식들을 익히고 가면 충분히 고득점을 노려볼 수 있음.\n\n- 학점은 항상 잘 주신다고 소문이 나 있음.\n\n- 예전에 전기회로1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안되어 집중을 못해서 잘 몰랐었는데..\n강의력이 정말정말 좋으심. 수업을 듣는데 흥미가 생김.\n- 과목 특성 상 고등학교 때 물리2 화학2 공부했다면 친숙한 내용임. 아니더라도 1학년 때 배우는 일반물리학/일반화학 내용이라 부담은 적음.\n양자역학도 배우긴 하지만 깊숙히 들어가지는 않아서 괜찮음.\n- 추천하고 싶은 강의임.","- 영강이지만 거의 반반으로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- ppt를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 

#### 6) 토크나이저 비교

In [58]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.


In [59]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab


In [ ]:
! bash install_mecab-ko_on_colab190912.sh

In [78]:
from konlpy.tag import Okt, Komoran, Hannanum, Kkma

def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "kkma":
        tokenizer = Kkma()
    else:
        tokenizer = Mecab()
    return tokenizer

In [79]:
klue_sample = list(klue['evaluation_spaced'])[0]

tokenizer_komo = get_tokenizer("komoran")  # komoran
print(tokenizer_komo.morphs(klue_sample), end = ' ')

['영강', '이', '지만', '거의', '반반', '으로', '한국어', '로', '다시', '설명', '하', '아', '주', '시', '어서', '수업', '을', '듣', '는', '데', '지장', '은', '없', '었', '음', '를', '사용', '하', '시', '어서', '수업', '하', '시', '었', '는데', '자료', '가', '깔끔', '하', 'ㅁ', '필요', '하', 'ㄴ', '공식', '유도', '를', '하', '아', '주', '시', '었', '는데', '단계', '별', '로', '차근차근', '이해', '를', '시키', '어', '주', '시', '어서', '정말', '좋', '았', '음', '친절', '하', '시', '고', '여러', '방면', '에서', '학생', '들', '입장', '을', '많이', '생각', '하', '아', '주', '시', '어서', '편하', '게', '수업', '을', '들', '을', '수', '있', '었', '음', '다만', '중간고사', '이후', '로', '양자역학', '파트', '에서', '내용', '이', '많', '다', '보니', '수업', '시간', '이', '길', '어', '지', '는', '경우', '가', '잦', '아', '짐', '코로나', '로', '인해', '비대', '이', '면', '강의', '이', '고', '녹화', '강의', '와', '실시간', '강의', '를', '섞', '어서', '수업', '하', '시', 'ㅁ', '중간고사', '이전', '에', '는', '하루', '는', '녹화', '강의', '하루', '는', '녹화', '강의', '절반', '강의', '에', '대하', 'ㄴ', '실시간', '질', '답', '시간', '절반', '으로', '수업', '하', '시', 'ㅁ', '매', '녹화', '강의', '마다', '출석', '체크', '가', '있', '고', '학습', '인정', '시간', '은', '녹화'

In [80]:
tokenizer_okt = get_tokenizer("okt")  # okt
print(tokenizer_okt.morphs(klue_sample), end = ' ')

['영강', '이지만', '거의', '반', '반', '으로', '한국어', '로', '다시', '설명', '해주셔서', '수업', '을', '듣는', '데', '지장', '은', '없었음', '를', '사용', '하셔서', '수업', '하셨는데', '자료', '가', '깔끔함', '필요한', '공식', '유도', '를', '해주셨는데', '단계', '별로', '차근차근', '이해', '를', '시켜주셔서', '정말', '좋았음', '친절하시고', '여러', '방면', '에서', '학생', '들', '입장', '을', '많이', '생각', '해', '주셔서', '편하게', '수업', '을', '들을', '수', '있었음', '다만', '중간고사', '이후', '로', '양자역학', '파트', '에서', '내용', '이', '많다', '보니', '수업', '시간', '이', '길어지는', '경우', '가', '잦아', '짐', '코로나', '로', '인해', '비대', '면', '강의', '이고', '녹화', '강의', '와', '실시간', '강의', '를', '섞어서', '수업', '하', '심', '중간고사', '이전', '에는', '하루', '는', '녹화', '강의', '하루', '는', '녹화', '강의', '절반', '강의', '에', '대한', '실시간', '질답', '시간', '절반', '으로', '수업', '하', '심', '매', '녹화', '강의', '마다', '출석', '체크', '가', '있고', '학습', '인정', '시간', '은', '녹화', '강의', '를', '풀', '로', '다', '봐야', '함', '당일', '에', '시청', '해야만', '출석', '되나', '가끔', '수업', '이', '늦어지는', '경우', '에는', '기한', '을', '늦춰', '주심', '배속', '기', '능', '을', '써도', '출석', '체크', '가', '되는지는', '모르겠음', '일정', '기한', '뒤', '에', '영상', '

In [81]:
tokenizer_kkma = get_tokenizer("kkma")  # kkma
print(tokenizer_kkma.morphs(klue_sample), end = ' ')

['영', '강', '이', '지만', '거의', '반반', '으로', '한국어', '로', '다시', '설명', '하', '어', '주', '시', '어서', '수업', '을', '듣', '는', '데', '지장', '은', '없', '었', '음', '를', '사용', '하', '시', '어서', '수업', '하', '시', '었', '는데', '자료', '가', '깔끔', '하', 'ㅁ', '필요', '하', 'ㄴ', '공식', '유도', '를', '하', '어', '주', '시', '었', '는데', '단계별', '로', '차근차근', '이해', '를', '시키', '어', '주', '시', '어서', '정말', '좋', '았', '음', '친절', '하', '시', '고', '여러', '방면', '에서', '학생', '들', '입장', '을', '많이', '생각', '하', '어', '주', '시', '어서', '편하', '게', '수업', '을', '듣', '을', '수', '있', '었', '음', '다', '만', '중간고사', '이후', '로', '양자', '역학', '파트', '에서', '내용', '이', '많', '다', '보', '니', '수업', '시간', '이', '긷', '어', '지', '는', '경우', '가', '잦', '아', '짐', '코로나', '로', '인하', '어', '비', '대면', '강의', '이', '고', '녹화', '강의', '와', '실시간', '강의', '를', '섞', '어서', '수업', '하심', '중간고사', '이전', '에', '는', '하루', '는', '녹화', '강의', '하루', '는', '녹화', '강', '의', '절반', '강의', '에', '대하', 'ㄴ', '실시간', '질', 'ㄹ', '답', '시간', '절반', '으로', '수업', '하심', '매', '녹화', '강의', '마다', '출석', '체크', '가', '있', '고', '학습', '인정', '시간', '은', '

In [82]:
# 토크나이저 육안으로 비교한 다음에 okt 토크나이저 사용하기로 결정
tokenizer_okt = get_tokenizer("okt")  # okt
evaluation_okt = []
for i in range(len(klue)):
  evaluation_okt.append(tokenizer_okt.morphs(list(klue['evaluation_spaced'])[i]))

klue['evaluation_okt'] = evaluation_okt
klue.head(3)

,class_name,professor_name,evaluation,evaluation_low,evaluation_dict,evaluation_kor,evaluation_spaced,evaluation_okt
0,물성전자공학(영강),김수진,"- 영강이지만 거의 5:5비율로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- PPT를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습문제에서 출제됨.\n교재를 찾아보면 같은 문제가 있을 것임.\n\n- 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나, 대부분의 문제가 객관식으로 출제됨.\n- 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효숫자에 신경쓰지 않고 풀 수 있음.\n- 풀이과정을 보시는지는 모르겠지만, 객관식이어서 각 답안이 맞으면 만점, 틀리면 0점임.\n- 주관식이나 서술형도 있지만 객관식이 대부분임.\n- 그러나 서술형 문제의 배점이 어마무시함. 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음.\n- 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분점수를 후하게 주시는 것 같음.\n- 중간고사 때는 중간고사 샘플자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음. 다만 기말고사에는 따로 없었음.\n- 나만 그런지는 모르겠으나,, 체감 난이도는 높았음. 과제보다는 난이도가 훨씬 쉽긴 함. 기말고사는 비대면 시험으로 진행되다 보니\n작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나, 체감 난이도는 높았음 ㅜ\n- 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면, 시험 기간에 따로 매달리지는 않아도 될 정도의 난이도로 나왔음.\n- 나머지는 수업자료나 과제에서 사용한 공식들을 익히고 가면 충분히 고득점을 노려볼 수 있음.\n\n- 학점은 항상 잘 주신다고 소문이 나 있음.\n\n- 예전에 전기회로1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안되어 집중을 못해서 잘 몰랐었는데..\n강의력이 정말정말 좋으심. 수업을 듣는데 흥미가 생김.\n- 과목 특성 상 고등학교 때 물리2 화학2 공부했다면 친숙한 내용임. 아니더라도 1학년 때 배우는 일반물리학/일반화학 내용이라 부담은 적음.\n양자역학도 배우긴 하지만 깊숙히 들어가지는 않아서 괜찮음.\n- 추천하고 싶은 강의임.","- 영강이지만 거의 5:5비율로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- ppt를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음.\n- 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습문제에서 출제됨.\n교재를 찾아보면 같은 문제가 있을 것임.\n\n- 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나, 대부분의 문제가 객관식으로 출제됨.\n- 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효숫자에 신경쓰지 않고 풀 수 있음.\n- 풀이과정을 보시는지는 모르겠지만, 객관식이어서 각 답안이 맞으면 만점, 틀리면 0점임.\n- 주관식이나 서술형도 있지만 객관식이 대부분임.\n- 그러나 서술형 문제의 배점이 어마무시함. 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음.\n- 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분점수를 후하게 주시는 것 같음.\n- 중간고사 때는 중간고사 샘플자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음. 다만 기말고사에는 따로 없었음.\n- 나만 그런지는 모르겠으나,, 체감 난이도는 높았음. 과제보다는 난이도가 훨씬 쉽긴 함. 기말고사는 비대면 시험으로 진행되다 보니\n작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나, 체감 난이도는 높았음 ㅜ\n- 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면, 시험 기간에 따로 매달리지는 않아도 될 정도의 난이도로 나왔음.\n- 나머지는 수업자료나 과제에서 사용한 공식들을 익히고 가면 충분히 고득점을 노려볼 수 있음.\n\n- 학점은 항상 잘 주신다고 소문이 나 있음.\n\n- 예전에 전기회로1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안되어 집중을 못해서 잘 몰랐었는데..\n강의력이 정말정말 좋으심. 수업을 듣는데 흥미가 생김.\n- 과목 특성 상 고등학교 때 물리2 화학2 공부했다면 친숙한 내용임. 아니더라도 1학년 때 배우는 일반물리학/일반화학 내용이라 부담은 적음.\n양자역학도 배우긴 하지만 깊숙히 들어가지는 않아서 괜찮음.\n- 추천하고 싶은 강의임.","- 영강이지만 거의 반반으로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음.\n- ppt를 사용하셔서 수업하셨는데 자료가 깔끔함.\n- 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음.\n- 친절하시고 여러 방면에서 학생들 입장을 많이 생각해주셔서 편하게 수업을 들을 수 있었음.\n- 다만 중간고사 이후로 양자역학 파트에서 내용이 많다보니 수업시간이 길어지는 경우가 잦아짐.\n\n- 코로나로 인해 비대면 강의이고, 녹화강의와 실시간강의를 섞어서 수업하심.\n- 중간고사 이전에는 하루는 녹화강의, 하루는 녹화강의 절반 + 강의에 대한 실시간 질답시간 절반으로 수업하심.\n- 매 녹화강의마다 출석체크가 있고 학습인정시간은 녹화강의를 풀로 다 봐야함.\n- 당일에 시청해야만 출석되나, 가끔 수업이 늦어지는 경우에는 기한을 늦춰주심.\n- 배속 기능을 써도 출석체크가 되는지는 모르겠음.\n- 일정 기한 뒤에 영상은 삭제하심. 기말고사 이전에 잠시 다 열어주셨음.\n\n- 과제는 매 단원이 끝날 때 쯤 나옴.\n- 각 단원 별 7문제 가량이 주어지지만, 한 학기 동안 4단원(중간 2단원, 기말 2단원)밖에 나가지 않기 때문에 과제에 대한 부담은 적음

In [83]:
del klue['evaluation']
del klue['evaluation_low']
del klue['evaluation_dict']
del klue['evaluation_kor']
klue.head(2)

,class_name,professor_name,evaluation_spaced,evaluation_okt
0,물성전자공학(영강),김수진,영강이지만 거의 반반으로 한국어로 다시 설명해주셔서 수업을 듣는 데 지장은 없었음 를 사용하셔서 수업하셨는데 자료가 깔끔함 필요한 공식 유도를 해주셨는데 단계별로 차근차근 이해를 시켜주셔서 정말 좋았음 친절하시고 여러 방면에서 학생들 입장을 많이 생각해 주셔서 편하게 수업을 들을 수 있었음 다만 중간고사 이후로 양자역학 파트에서 내용이 많다 보니 수업 시간 이 길어지는 경우가 잦아 짐 코로나로 인해 비대면 강의이고 녹화강의와 실시간 강의를 섞어서 수업하심 중간고사 이전에는 하루는 녹화강의 하루는 녹화강의 절반 강의에 대한 실시간 질답시간 절반으로 수업 하심 매 녹화강의마다 출석 체크가 있고 학습 인정시간은 녹화강의를 풀로 다 봐야 함 당일에 시청 해야만 출석되나 가끔 수업이 늦어지는 경우에는 기한을 늦춰 주심 배속 기 능을 써도 출석 체크가 되는지는 모르겠음 일정 기한 뒤에 영상은 삭제하심 기말고사 이전에 잠시 다 열어주셨음 과제는 매 단원이 끝날 때 쯤 나옴 각 단원 별 7문제 가량이 주어지지만 한 학기 동안 4단원 중간 2단원 기말 2단원밖에 나가지 않기 때문에 과제에 대한 부담은 적음 별도로 언급되지는 않지만 대부분의 문제는 교재 내 연습 문제에서 출제됨 교재를 찾아보 면 같은 문제가 있을 것임 코로나 특수로 이번 시험이 특별했던 것인지는 모르겠으나 대부분의 문제가 객관식으로 출제됨 객관식으로 출제되므로 계산기에 값을 잘못 입력했을 때 오답을 적지 않을 수 있으며 사실상 단위나 유효 숫자에 신경 쓰지 않고 풀 수 있음 풀이 과정을 보시는지는 모르겠지만 객관식이어서 각 답안이 맞으면 만점 틀리면 0점 임 주관식이나 서술형 도 있 지만 객관식이 대부분 임 그러나 서술형 문제의 배점이 어 마무 시 함 중간고사와 기말고사에 한 문제씩 나왔는데 배점이 다른 문제에 비해서 월등히 높음 중간고사 때 엉터리로 쓴 것 같은데 점수를 잘 받은 것을 보면 부분 점수를 후하게 주시는 것 같음 중간고사 때는 중간고사 샘플 자료를 블랙보드에 올려주셔서 중간고사를 대비할 수 있게 해주셨음 다만 기 말고사에는 따로 없었음 나만 그런지는 모르겠으나 체감 난이도는 높았음 과 제보다는 난이도가 훨씬 쉽긴 함 기 말고사는 비대면 시험으로 진행되다 보니 작정하고 다른 학기에 비해 쉽게 냈다고 하셨으나 체감 난이도는 높았음 공식 유도나 증명에 대한 문제도 나오지만 수업 시간에 집중해서 듣고 필기하면 시험 기간에 따로 매달리지는 않아도 될 정도의 난이 도로 나왔음 나머지는 수업 자료 나 과제에서 사용 한 공식들을 익히고 가면 충분히 고득점을 노려 볼 수 있음 학점은 항상 잘 주신다고 소문이 나 있음 예전에 전기회로 1 수업을 대면으로 들었을 때는 뒷자리에 앉으면 목소리가 잘 전달이 안 되어 집중을 못해서 잘 몰랐었는데 강의력이 정말 정말 좋으심 수업을 듣는데 흥미가 생김 과목 특성 상 고등학교 때 물리2 화학 2 공부했다면 친숙한 내용 임 아니더 라도 일학년 때 배우는 일반 물리학 일반화학 내용이라 부담은 적음양자역학도 배우 긴 하지만 깊숙히 들어가지는 않아서 괜찮음 추천하고 싶은 강의임,"[영강, 이지만, 거의, 반, 반, 으로, 한국어, 로, 다시, 설명, 해주셔서, 수업, 을, 듣는, 데, 지장, 은, 없었음, 를, 사용, 하셔서, 수업, 하셨는데, 자료, 가, 깔끔함, 필요한, 공식, 유도, 를, 해주셨는데, 단계, 별로, 차근차근, 이해, 를, 시켜주셔서, 정말, 좋았음, 친절하시고, 여러, 방면, 에서, 학생, 들, 입장, 을, 많이, 생각, 해, 주셔서, 편하게, 수업, 을, 들을, 수, 있었음, 다만, 중간고사, 이후, 로, 양자역학, 파트, 에서, 내용, 이, 많다, 보니, 수업, 시간, 이, 길어지는, 경우, 가, 잦아, 짐, 코로나, 로, 인해, 비대, 면, 강의, 이고, 녹화, 강의, 와, 실시간, 강의, 를, 섞어서, 수업, 하, 심, 중간고사, 이전, 에는, 하루, 는, 녹화, 강의, ...]"
1,물성전자공학(영강),김수진,김수진 교수님의 물성 전자공학은 사이버 강의였음에도 강의력이 좋으셔서 그런지 수월하게 했습니다 과제는 34번 정도 나왔고 시험은 중간고사 때 조금 생각해야 하는 문제 하나가 있는 것을 제외하면 전반적으로 쉬웠습니다 선택하면 후회는 없는 교수님이라고 생각합니다 총 평쁠 20점 정도로 좋은 학점 받았는데 적당히 평쁠하면 받을 수 있는 걸로 알고 있습니다 이 강 의를 들으면서 이번에 들은 강의 중에 제일 재밌고 유익하다고 생각했습니다,"[김수진, 교수, 님, 의, 물성, 전자공학, 은, 사이버, 강의, 였음에도, 강의, 력, 이, 좋으셔서, 그런지, 수월하게, 했습니다, 과제, 는, 34, 번, 정도, 나왔고, 시험, 은, 중간고사, 때, 조금, 생각, 해야, 하는, 문제, 하나, 가, 있는, 것, 을, 제외, 하면, 전반, 적, 으로, 쉬웠습니다, 선택, 하면, 후회, 는, 없는, 교수, 님, 이라고, 생각, 합니다, 총, 평쁠, 20, 점, 정도, 로, 좋은, 학점, 받았는데, 적당히, 평쁠하, 면, 받을, 수, 있는, 걸, 로, 알, 고, 있습니다, 이, 강, 의, 를, 들으면서, 이번, 에, 들은, 강의, 중, 에, 제일, 재밌고, 유익하다고, 생각, 했습니다]"


In [88]:
klue.to_csv('/content/drive/MyDrive/klue_dataset/preprocessed.csv')